In [ ]:
from vllm import LLM, SamplingParams

llm = LLM(model="/root/autodl-tmp/data/model_ckpts/Qwen/Qwen3-8B")

/root/miniconda3/envs/xi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 08-08 15:57:52 [__init__.py:235] Automatically detected platform cuda.
WARNING 08-08 15:58:03 [config.py:3392] Your device 'Tesla V100-PCIE-32GB' (with compute capability 7.0) doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 08-08 15:58:03 [config.py:3443] Casting torch.bfloat16 to torch.float16.
INFO 08-08 15:58:03 [config.py:1604] Using max model len 40960
WARNING 08-08 15:58:03 [arg_utils.py:1690] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
WARNING 08-08 15:58:03 [arg_utils.py:1486] Chunked prefill is enabled by default for models with max_model_len > 32K. Chunked prefill might not work with some features or models. If you encounter any issues, please disable by launching with --enable-chunked-prefill=False.


2025-08-08 15:58:03,784	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 08-08 15:58:03 [config.py:2434] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 08-08 15:58:03 [llm_engine.py:228] Initializing a V0 LLM engine (v0.10.0) with config: model='/root/autodl-tmp/data/model_ckpts/Qwen/Qwen3-8B', speculative_config=None, tokenizer='/root/autodl-tmp/data/model_ckpts/Qwen/Qwen3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=40960, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detailed_t

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  20% Completed | 1/5 [00:20<01:22, 20.52s/it]
Loading safetensors checkpoint shards:  40% Completed | 2/5 [00:41<01:03, 21.08s/it]
Loading safetensors checkpoint shards:  60% Completed | 3/5 [01:03<00:42, 21.15s/it]
Loading safetensors checkpoint shards:  80% Completed | 4/5 [01:18<00:18, 18.95s/it]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [01:25<00:00, 14.66s/it]
Loading safetensors checkpoint shards: 100% Completed | 5/5 [01:25<00:00, 17.17s/it]


INFO 08-08 15:59:33 [default_loader.py:262] Loading weights took 85.91 seconds


INFO 08-08 15:59:34 [model_runner.py:1115] Model loading took 15.2683 GiB and 86.185500 seconds
INFO 08-08 15:59:35 [worker.py:295] Memory profiling takes 0.99 seconds
INFO 08-08 15:59:35 [worker.py:295] the current vLLM instance can use total_gpu_memory (31.74GiB) x gpu_memory_utilization (0.90) = 28.57GiB
INFO 08-08 15:59:35 [worker.py:295] model weights take 15.27GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 1.40GiB; the rest of the memory reserved for KV Cache is 11.81GiB.
INFO 08-08 15:59:36 [executor_base.py:113] # cuda blocks: 5374, # CPU blocks: 1820
INFO 08-08 15:59:36 [executor_base.py:118] Maximum concurrency for 40960 tokens per request: 2.10x
INFO 08-08 15:59:39 [model_runner.py:1385] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider de

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:24<00:00,  1.40it/s]

INFO 08-08 16:00:04 [model_runner.py:1537] Graph capturing finished in 25 secs, took 1.29 GiB
INFO 08-08 16:00:04 [llm_engine.py:424] init engine (profile, create kv cache, warmup model) took 30.12 seconds


AttributeError: 'LLM' object has no attribute 'get_embeddings'

In [1]:
question = "Question: In a population of giraffes, an environmental change occurs that favors individuals that are tallest. As a result, more of the taller individuals are able to obtain nutrients and survive to pass along their genetic information. This is an example of\nA. directional selection.\nB. stabilizing selection.\nC. sexual selection.\nD. disruptive selection.\n\nAnswer: "
qa = [f"{question}_{answer}." for answer in ["A", "B", "C", "D"]]

In [ ]:
from vllm.engine.arg_utils import EngineArgs
from vllm.engine.llm_engine import LLMEngine
import torch

# 直接使用LLMEngine而非高级LLM接口
engine_args = EngineArgs(model="/root/autodl-tmp/data/model_ckpts/Qwen/Qwen3-8B")
engine = LLMEngine.from_engine_args(engine_args)

In [4]:
def get_pad_token_id(engine):
    # 方法1: 从底层tokenizer获取
    if hasattr(engine.tokenizer, "tokenizer"):  # 访问实际的HF tokenizer
        if hasattr(engine.tokenizer.tokenizer, "pad_token_id"):
            return engine.tokenizer.tokenizer.pad_token_id
    
    # 方法2: 使用通用默认值
    return 0  # 很多模型使用0作为padding token

def get_embeddings_efficiently(texts, batch_size=32):
    all_embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        
        # 获取tokenized输入
        token_ids_list = [engine.tokenizer.encode(text) for text in batch]
        max_len = max(len(ids) for ids in token_ids_list)
        
        # 填充到相同长度
        padded_ids = [ids + [engine.tokenizer.tokenizer.pad_token_id] * (max_len - len(ids)) for ids in token_ids_list]
        input_ids = torch.tensor(padded_ids).cuda()
        
        # 直接访问embedding层，避免全部前向传播
        embedding_layer = engine.model_executor.model.get_input_embeddings()
        with torch.no_grad():
            batch_embeddings = embedding_layer(input_ids)
        
        all_embeddings.append(batch_embeddings)
    
    return torch.cat(all_embeddings, dim=0)

for _ in range(40):
    # 使用高效的批处理方式获取所有嵌入
    all_embeddings = get_embeddings_efficiently(qa, batch_size=4)

AttributeError: 'UniProcExecutor' object has no attribute 'model'

In [ ]:
from vllm import LLM, SamplingParams

llm = LLM(model="/root/autodl-tmp/data/model_ckpts/Qwen/Qwen3-8B", task="embed")

In [10]:
llm.embed(qa)

ValueError: Embedding API is not supported by this model. Please set `--task embed`.

In [6]:
def get_last_layer_output(text, llm):
    # 获取内部模型
    model = llm.llm_engine.model_executor.worker.model
    
    # 注册一个钩子来捕获最后一层输出
    last_hidden_states = []
    
    def hook(module, input, output):
        last_hidden_states.append(output[0].detach().cpu())
    
    # 为最后一层注册钩子
    # Llama和大多数模型的最后一层通常在这个位置
    handle = model.layers[-1].register_forward_hook(hook)
    
    try:
        # 简单运行一个生成，触发模型前向传播
        outputs = llm.generate([text], max_tokens=1)
        
        # 钩子已经捕获了最后一层的输出
        return last_hidden_states[0]
    finally:
        # 删除钩子，防止内存泄漏
        handle.remove()

text = "这是一个测试文本"
hidden_state = get_last_layer_output(text, llm)

# 4. 查看结果
print(f"Hidden state shape: {hidden_state.shape}")
print(f"Sample values: {hidden_state[0, :5]}")  # 打印前几个值

AttributeError: 'UniProcExecutor' object has no attribute 'worker'

In [7]:
print("LLM结构:")
for attr in dir(llm):
    if not attr.startswith('_'):
        print(f"- {attr}")

print("\nLLM Engine结构:")
for attr in dir(llm.llm_engine):
    if not attr.startswith('_'):
        print(f"- {attr}")

print("\nModel Executor结构:")
for attr in dir(llm.llm_engine.model_executor):
    if not attr.startswith('_'):
        print(f"- {attr}")

# 如果是多进程执行器，可能需要检查worker
if hasattr(llm.llm_engine.model_executor, "worker"):
    print("\nWorker结构:")
    for attr in dir(llm.llm_engine.model_executor.worker):
        if not attr.startswith('_'):
            print(f"- {attr}")

# 对于最新版本可能是driver
if hasattr(llm.llm_engine.model_executor, "driver"):
    print("\nDriver结构:")
    for attr in dir(llm.llm_engine.model_executor.driver):
        if not attr.startswith('_'):
            print(f"- {attr}")

LLM结构:
- DEPRECATE_LEGACY
- apply_model
- beam_search
- chat
- classify
- collective_rpc
- default_sampling_params
- deprecate_legacy_api
- embed
- encode
- engine_class
- generate
- get_default_sampling_params
- get_metrics
- get_tokenizer
- llm_engine
- request_counter
- reset_prefix_cache
- score
- set_tokenizer
- sleep
- start_profile
- stop_profile
- wake_up

LLM Engine结构:
- DO_VALIDATE_OUTPUT
- abort_request
- add_logger
- add_lora
- add_request
- async_callbacks
- cache_config
- cached_scheduler_outputs
- check_health
- collective_rpc
- create_trace_span
- decoding_config
- detokenizer
- device_config
- do_log_stats
- do_tracing
- enable_output_validation
- from_engine_args
- from_vllm_config
- generation_config_fields
- get_decoding_config
- get_lora_config
- get_model_config
- get_num_unfinished_requests
- get_parallel_config
- get_scheduler_config
- get_tokenizer
- get_tokenizer_group
- get_vllm_config
- has_unfinished_requests
- has_unfinished_requests_for_virtual_engine
- i